In [1]:
pip install pandas dash

In [22]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

spacex_df_grouped = spacex_df.groupby(['Launch Site', 'class'])['class'].count()
spacex_df_grouped = spacex_df_grouped.to_frame()
spacex_df_grouped = spacex_df_grouped.rename(columns = {'class' : 'count'})
spacex_df_grouped.columns.name = None
spacex_df_grouped = spacex_df_grouped.reset_index(drop = False)
spacex_df_grouped

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                            ],
                                            value='ALL',
                                            placeholder="Select a launch site here",
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       1000: '1000',
                                                      2000: '2000',
                                                      3000: '3000',
                                                      4000: '4000',
                                                      5000: '5000',
                                                      6000: '6000',
                                                      7000: '7000',
                                                      8000: '8000',
                                                      9000: '9000',
                                                      10000: '10000'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    #df =  airline_data[airline_data['Year']==int(entered_year)]
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        filtered_df = spacex_df_grouped[spacex_df_grouped['Launch Site'] == str(entered_site)]
        fig_1 = px.pie(filtered_df, values='count', 
        names ='class', 
        title = f'Total Success Launches By: {entered_site}')
        return fig_1
        # return the outcomes piechart for a selected site

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property='value')])
def get_scatter_chart(entered_site, entered_payload):
    #df =  airline_data[airline_data['Year']==int(entered_year)]
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', 
        y='class', color = 'Booster Version Category',
        title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        filtered_df_1 = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df_2 = filtered_df_1[(filtered_df_1['Payload Mass (kg)'] >= entered_payload[0]) & (filtered_df_1['Payload Mass (kg)'] <= entered_payload[1])]
        fig_1 = px.scatter(filtered_df_2, x='Payload Mass (kg)', 
        y ='class', color = 'Booster Version Category',
        title = f'Correlation between Payload and Success for: {entered_site}')
        return fig_1
        # return the outcomes piechart for a selected site


# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Jun/2023 17:16:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 17:16:10] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 17:16:10] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 17:16:11] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 17:16:11] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 17:16:11] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 17:16:11] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 17:16:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 17:16:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 17:16:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 17:16:13] "POST /_dash-